<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"></ul></div>

# Импорт библиотек и загрузка данных

In [23]:
# Данный python-скрипт имитирует запрос к БД
# Напишите ваш SQL-запрос в query и запустите данный python-скрипт для получения результата
# Перед запуском скрипта установите библиотеку duckdb

# Установка библиотеки duckdb
#!pip install duckdb duckdb-engine

# Импорт библиотек
import pandas as pd
import duckdb
import datetime as dt
import numpy as np


In [2]:
# Задание таблиц БД
users = pd.read_csv('users.csv')
course_users = pd.read_csv('course_users.csv')
courses = pd.read_csv('courses.csv')
course_types = pd.read_csv('course_types.csv')
lessons = pd.read_csv('lessons.csv')
subjects = pd.read_csv('subjects.csv')
cities = pd.read_csv('cities.csv')
homework_done = pd.read_csv('homework_done.csv')
homework = pd.read_csv('homework.csv')
homework_lessons = pd.read_csv('homework_lessons.csv')
user_roles = pd.read_csv('user_roles.csv') 



In [3]:
# Задание SQL-запроса
query = """
--- выведем ID, фамилию пользователей с ролью 'student' и название города
WITH students AS (SELECT DISTINCT users.id AS user_id, users.last_name, cities.name AS city, COUNT(homework_done.id) AS homeworks  
              FROM users
              LEFT JOIN user_roles ON users.user_role_id=user_roles.id
              LEFT JOIN cities ON users.city_id=cities.id 
              JOIN homework_done ON homework_done.user_id=users.id
              WHERE user_roles.name='student'
              GROUP BY users.id, users.last_name, cities.name),
--- выведем ID, название и тип курса, число уроков в месяц на курсе, дата старта курса,
--- предмет, тип предмета для годовых курсов ОГЭ и ЕГЭ             
      year_courses AS (SELECT courses.id AS course_id, courses.name AS course_name, course_types.name AS type_course,
               courses.lessons_in_month, CAST(courses.starts_at AS DATE) AS starts_at, subjects.name AS subject_name, subjects.project
               FROM courses
               JOIN course_types ON course_types.id=courses.course_type_id
               JOIN subjects ON subjects.id=courses.subject_id
               WHERE course_types.name='Годовой')  
--- считаю, что дата создания записи в таблице "Связка курсов и пользователей" - это и есть
--- дата открытия курса ученику. А количество полных месяцев курса можно рассчитать 
--- делением числа открытых ученику уроков на число уроков в месяц на курсе               
SELECT year_courses.course_id, year_courses.course_name, year_courses.subject_name, year_courses.project,
       year_courses.type_course, year_courses.starts_at, students.user_id, students.last_name, students.city,
       course_users.active, CAST(course_users.created_at AS DATE) AS created_at,
       FLOOR(course_users.available_lessons/year_courses.lessons_in_month) AS months, students.homeworks
   
FROM students
JOIN course_users ON course_users.user_id=students.user_id
JOIN year_courses ON year_courses.course_id=course_users.course_id
                

"""


In [4]:
# Выполнение SQL-запроса
df = duckdb.query(query).to_df()


In [5]:
# Вывод результата
display(df)

,course_id,course_name,subject_name,project,type_course,starts_at,user_id,last_name,city,active,created_at,months,homeworks
0,16,Годовой 2к25 стандарт,Обществознание,ЕГЭ,Годовой,2024-09-16,10001,Markova,Самара,1,2024-07-25,2.0,22
1,19,Годовой 2к25 стандарт,Русский,ЕГЭ,Годовой,2024-09-21,10001,Markova,Самара,1,2024-07-25,2.0,22
2,24,В погоне за пятеркой 2К25 стандарт,Математика (база),ЕГЭ,Годовой,2024-09-17,10001,Markova,Самара,1,2024-09-01,2.0,22
3,17,Годовой 2к25 стандарт,Английский язык,ЕГЭ,Годовой,2024-09-20,10003,Драгомирецкий,None,1,2024-09-05,0.0,2
4,16,Годовой 2к25 стандарт,Обществознание,ЕГЭ,Годовой,2024-09-16,10006,Пушкина,Москва,1,2024-07-25,2.0,17
...,...,...,...,...,...,...,...,...,...,...,...,...,...
5391,19,Годовой 2к25 стандарт,Русский,ЕГЭ,Годовой,2024-09-21,33719,Новьюхова,Тюмень,1,2024-10-11,2.0,23
5392,18,Годовой 2к25 стандарт,История,ЕГЭ,Годовой,2024-09-18,33719,Новьюхова,Тюмень,1,2024-10-11,2.0,23
5393,16,Годовой 2к25 стандарт,Обществознание,ЕГЭ,Годовой,2024-09-16,33883,Храмова,Самара,1,2024-10-11,2.0,19
5394,19,Годовой 2к25 стандарт,Русский,ЕГЭ,Годовой,2024-09-21,33883,Храмова,Самара,1,2024-10-11,2.0,19


# Предобработка данных

In [6]:
# посмотрим общую информацию о датасете
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5396 entries, 0 to 5395
Data columns (total 13 columns):
 #   Column        Non-Null Count  Dtype         
---  ------        --------------  -----         
 0   course_id     5396 non-null   int64         
 1   course_name   5396 non-null   object        
 2   subject_name  5396 non-null   object        
 3   project       5396 non-null   object        
 4   type_course   5396 non-null   object        
 5   starts_at     5396 non-null   datetime64[us]
 6   user_id       5396 non-null   int64         
 7   last_name     5396 non-null   object        
 8   city          5109 non-null   object        
 9   active        5396 non-null   int64         
 10  created_at    5396 non-null   datetime64[us]
 11  months        5383 non-null   float64       
 12  homeworks     5396 non-null   int64         
dtypes: datetime64[us](2), float64(1), int64(4), object(6)
memory usage: 548.2+ KB


Проверим наличие пропущенных значений и их долю 

In [7]:
report = df.isna().sum().to_frame()
report = report.rename(columns = {0: 'missing_values'})
report['% of total'] = (report['missing_values'] / df.shape[0]).round(2)
report.sort_values(by = 'missing_values', ascending = False)

,missing_values,% of total
city,287,0.05
months,13,0.00
course_id,0,0.00
course_name,0,0.00
subject_name,0,0.00
project,0,0.00
type_course,0,0.00
starts_at,0,0.00
user_id,0,0.00
last_name,0,0.00


Так как доля пропусков мала, удалим данные строки из датафрейма

In [8]:
df = df.dropna(subset = ['months', 'city'])

In [9]:
df.isna().sum()

course_id       0
course_name     0
subject_name    0
project         0
type_course     0
starts_at       0
user_id         0
last_name       0
city            0
active          0
created_at      0
months          0
homeworks       0
dtype: int64

Изменим тип данных в столбце months "Число полных месяцев курсов открыто у ученика" на тип целых чисел int       

In [10]:
df['months'] = df['months'].astype(int)

Проверим наличие дубликатов

In [11]:
df.duplicated().sum()

0

Посмотрим уникальные значения городов

In [12]:
df['city'].unique()


array(['Самара', 'Москва', 'Якутск', 'Анива', 'Ижевск', 'Краснодар',
       'Хабаровск', 'Отрадный', 'Донецк', 'Холмск', 'Туймазы',
       'Ханты-Мансийск', 'Йошкар-Ола', 'Екатеринбург', 'Ангарск', 'Арск',
       'Пермь', 'Бабаево', 'Курган', 'Брянск', 'Ростов-на-Дону',
       'Ессентуки', 'Куйбышев', 'Боровичи', 'Нефтеюганск', 'Коркино',
       'Тольятти', 'Красноярск', 'Салават', 'Янаул', 'Владивосток',
       'Новый Уренгой', 'Сочи', 'Симферополь', 'Красногорск',
       'Набережные Челны', 'Шебекино', 'Абакан', 'Сафоново',
       'Севастополь', 'Гатчина', 'Корсаков', 'Каспийск', 'Баксан',
       'Барнаул', 'Псков', 'Кондопога', 'Волгоград', 'Кемерово',
       'Улан-Удэ', 'Чапаевск', 'Новороссийск', 'Ярославль', 'Южа',
       'Химки', 'Санкт-Петербург', 'Волгодонск', 'Ульяновск',
       'Первоуральск', 'Нижний Тагил', 'Моршанск', 'Элиста',
       'Прокопьевск', 'Орёл', 'Сестрорецк', 'Рязань', 'Павлово', 'Омск',
       'Камышлов', 'Касли', 'Маркс', 'Серпухов', 'Мытищи', 'Кремёнки',
  

In [13]:
df.describe()

,course_id,starts_at,user_id,active,created_at,months,homeworks
count,5096.000000,5096,5096.000000,5096.000000,5096,5096.000000,5096.000000
mean,21.151099,2024-09-19 03:03:57.362637,24725.202904,0.966248,2024-09-17 17:25:48.508634,2.291209,21.641680
min,16.000000,2024-09-15 00:00:00,10001.000000,0.000000,2024-07-25 00:00:00,0.000000,1.000000
25%,16.000000,2024-09-16 00:00:00,25005.750000,1.000000,2024-09-16 00:00:00,2.000000,11.000000
50%,19.000000,2024-09-17 00:00:00,28304.500000,1.000000,2024-09-25 00:00:00,2.000000,18.000000
75%,23.000000,2024-09-21 00:00:00,30575.500000,1.000000,2024-10-06 00:00:00,2.000000,31.000000
max,81.000000,2024-10-15 00:00:00,33999.000000,1.000000,2024-11-09 00:00:00,10.000000,82.000000
std,7.266383,NaN,8559.682921,0.180608,NaN,1.704687,14.444967


In [14]:
df[df['active'] ==0]

,course_id,course_name,subject_name,project,type_course,starts_at,user_id,last_name,city,active,created_at,months,homeworks
8,16,Годовой 2к25 стандарт,Обществознание,ЕГЭ,Годовой,2024-09-16,10011,Ищенко,Анива,0,2024-07-26,0,11
9,24,В погоне за пятеркой 2К25 стандарт,Математика (база),ЕГЭ,Годовой,2024-09-17,10011,Ищенко,Анива,0,2024-07-26,0,11
14,16,Годовой 2к25 стандарт,Обществознание,ЕГЭ,Годовой,2024-09-16,10014,Павлова,Краснодар,0,2024-07-26,0,4
21,25,Годовой 2к25 стандарт,Биология,ЕГЭ,Годовой,2024-09-17,10019,Кутылина,Отрадный,0,2024-07-26,1,27
25,22,Годовой 2к25 стандарт,Литература,ЕГЭ,Годовой,2024-09-17,10024,Имамутдинова,Туймазы,0,2024-07-26,0,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...
5090,16,Годовой 2к25 стандарт,Обществознание,ЕГЭ,Годовой,2024-09-16,33757,Савченко,Кропоткин,0,2024-10-11,1,7
5119,24,В погоне за пятеркой 2К25 стандарт,Математика (база),ЕГЭ,Годовой,2024-09-17,33796,Манохина,Иркутск,0,2024-10-11,1,8
5322,42,Годовой курс 2к25,Биология ОГЭ,ОГЭ,Годовой,2024-09-30,33692,Беличева,Тюмень,0,2024-10-11,0,12
5343,16,Годовой 2к25 стандарт,Обществознание,ЕГЭ,Годовой,2024-09-16,33900,Ощепков,Чернушка,0,2024-10-11,0,8


# Вывод:  
перед нами данные об учениках с годовых курсов ЕГЭ и ОГЭ. Проведена проверка на наличие дубликатов, пропущенных значений. В датафрейме встречаются строки с одинаковыми значениями id клиента, так как эти пользователи проходят несколько курсов одновременно. Наличие пропущенных значений в столбце months связано с отсутствием информации об открытых уроках у данного пользователя. Пропуски в столбце city связаны, скорее всего, из-за ошибки ввода данных пользователя при регистрации. 
Из 5096 строк - в 172 случаях ученики отчисленные с курса. Самая ранняя дата старта курса в датасете - 15.09.2024, максимальная - 15.10.2024. 

# Категоризация

Поделим всех учеников на волны

    Критерии волн:
    • 0 волна – присоединились на курс до даты старта курса включительно
    • 1 волна - присоединились на курс в течение 1 недели после даты старта курса включительно
    • 2 волна - присоединились на курс в течение от 1 до 2 недель включительно после даты старта курса
    • 3 волна - присоединились на курс в течение от 2 до 3 недель включительно после даты старта курса
    • 4 волна - присоединились на курс в течение от 3 до 4 недель включительно после даты старта курса
    • 5 волна - присоединились на курс от 4 недель после даты старта курса

In [26]:
df['delta_week'] = df['created_at'] - df['starts_at']
df['delta_week'] = df['delta_week']/np.timedelta64(1,'W')

In [32]:
df.head()

,course_id,course_name,subject_name,project,type_course,starts_at,user_id,last_name,city,active,created_at,months,homeworks,delta_week
0,16,Годовой 2к25 стандарт,Обществознание,ЕГЭ,Годовой,2024-09-16,10001,Markova,Самара,1,2024-07-25,2,22,-7.571429
1,19,Годовой 2к25 стандарт,Русский,ЕГЭ,Годовой,2024-09-21,10001,Markova,Самара,1,2024-07-25,2,22,-8.285714
2,24,В погоне за пятеркой 2К25 стандарт,Математика (база),ЕГЭ,Годовой,2024-09-17,10001,Markova,Самара,1,2024-09-01,2,22,-2.285714
4,16,Годовой 2к25 стандарт,Обществознание,ЕГЭ,Годовой,2024-09-16,10006,Пушкина,Москва,1,2024-07-25,2,17,-7.571429
5,16,Годовой 2к25 стандарт,Обществознание,ЕГЭ,Годовой,2024-09-16,10010,Мирзоян,Якутск,1,2024-07-26,2,42,-7.428571


In [29]:
# Создадим функцию для категоризации данных по волнам
def category(row):
    if df['delta_week']<=0:
        return '0 волна'
    if 0<df['delta_week']<=1:
        return '1 волна'
    if 1<df['delta_week']<=2:
        return '2 волна'
    if 2<df['delta_week']<=3:
        return '3 волна' 
    if 3<df['delta_week']<=4:
        return '4 волна' 
    else:
        return '5 волна'

In [30]:
# добавим в датафрейм df столбец 'flow' со значениями-результатами работы функции 
df['flow'] = df.apply(category, axis=1)

ValueError: The truth value of a Series is ambiguous. Use a.empty, a.bool(), a.item(), a.any() or a.all().